In [14]:
# ============================================================
# IdleCleaner — Automated Idle Cloud Resource Cleanup Simulator
# ============================================================

import os
import json
import random
from datetime import datetime, timedelta, timezone

# Configuration
RESOURCES_FILE = "data/cloud_resources.json"
CLEANUP_REPORT = "data/cleanup_report.json"
IDLE_THRESHOLD_DAYS = 30

# Ensure directories
os.makedirs(os.path.dirname(RESOURCES_FILE), exist_ok=True)
os.makedirs(os.path.dirname(CLEANUP_REPORT), exist_ok=True)

# Generate dummy cloud-resources if not exists
if not os.path.exists(RESOURCES_FILE):
    resources = []
    now = datetime.now(timezone.utc)
    for i in range(1, 51):
        created_days_ago = random.randint(1, 120)
        last_used_days_ago = random.randint(0, 90)
        resources.append({
            "resource_id": f"res-{i:03}",
            "type": random.choice(["vm_instance", "storage_bucket", "database"]),
            "created_at": (now - timedelta(days=created_days_ago)).isoformat(),
            "last_used_at": (now - timedelta(days=last_used_days_ago)).isoformat(),
            "status": random.choice(["active", "stopped", "terminated"])
        })
    with open(RESOURCES_FILE, "w", encoding="utf-8") as f:
        json.dump(resources, f, indent=2)

def load_resources(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def evaluate_idle(resources, threshold_days):
    now = datetime.now(timezone.utc)
    report = {
        "timestamp": now.isoformat(),
        "total_resources": len(resources),
        "idle_candidates": 0,
        "cleanup_actions": [],
    }
    for res in resources:
        last_used = datetime.fromisoformat(res["last_used_at"])
        idle_days = (now - last_used).days
        if idle_days > threshold_days and res["status"] == "stopped":
            report["idle_candidates"] += 1
            report["cleanup_actions"].append({
                "resource_id": res["resource_id"],
                "type": res["type"],
                "idle_days": idle_days,
                "action": "eligible_for_cleanup"
            })
    return report

def run_cleanup_audit(input_path, output_path, threshold_days):
    resources = load_resources(input_path)
    report = evaluate_idle(resources, threshold_days)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(report, f, indent=2)
    return report

if __name__ == "__main__":
    result = run_cleanup_audit(RESOURCES_FILE, CLEANUP_REPORT, IDLE_THRESHOLD_DAYS)
    print("✅ Cleanup audit complete:", result)


✅ Cleanup audit complete: {'timestamp': '2025-10-19T10:31:28.190105+00:00', 'total_resources': 50, 'idle_candidates': 5, 'cleanup_actions': [{'resource_id': 'res-006', 'type': 'database', 'idle_days': 32, 'action': 'eligible_for_cleanup'}, {'resource_id': 'res-015', 'type': 'storage_bucket', 'idle_days': 88, 'action': 'eligible_for_cleanup'}, {'resource_id': 'res-021', 'type': 'vm_instance', 'idle_days': 47, 'action': 'eligible_for_cleanup'}, {'resource_id': 'res-041', 'type': 'vm_instance', 'idle_days': 46, 'action': 'eligible_for_cleanup'}, {'resource_id': 'res-049', 'type': 'vm_instance', 'idle_days': 78, 'action': 'eligible_for_cleanup'}]}
